In [1]:
import pandas as pd
import win32com.client
import time

In [3]:

instCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
print(instCpCybos.IsConnect)

0


# 60일 동안의 거래량 구하기

In [5]:
# Create object
instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")

# SetInputValue
instStockChart.SetInputValue(0, "A035420")
instStockChart.SetInputValue(1, ord('2'))
instStockChart.SetInputValue(4, 60) # 60일
instStockChart.SetInputValue(5, 8) # 거래량
instStockChart.SetInputValue(6, ord('D'))
instStockChart.SetInputValue(9, ord('1'))

# BlockRequest
instStockChart.BlockRequest()

# GetData
volumes = []
numData = instStockChart.GetHeaderValue(3)
for i in range(numData):
    volume = instStockChart.GetDataValue(0, i)
    volumes.append(volume)
# print(volumes)



# Calculate average volume
averageVolume = (sum(volumes) - volumes[0]) / (len(volumes) -1)

if(volumes[0] > averageVolume * 10):
    print("대박주")
else:
    print("일반주", volumes[0] / averageVolume)

일반주 0.7157689096256812


# 거래량 체크하는 함수

In [6]:

def CheckVolumn(instStockChart, code):
    # SetInputValue
    instStockChart.SetInputValue(0, code)
    instStockChart.SetInputValue(1, ord('2'))
    instStockChart.SetInputValue(4, 60)
    instStockChart.SetInputValue(5, 8)
    instStockChart.SetInputValue(6, ord('D'))
    instStockChart.SetInputValue(9, ord('1'))

    # BlockRequest
    instStockChart.BlockRequest()

    # GetData
    volumes = []
    numData = instStockChart.GetHeaderValue(3)
    for i in range(numData):
        volume = instStockChart.GetDataValue(0, i)
        volumes.append(volume)

    # Calculate average volume
    averageVolume = (sum(volumes) - volumes[0]) / (len(volumes) -1)

    if(volumes[0] > averageVolume * 10):
        return 1
    else:
        return 0

In [9]:
# 종목별 코드 리스트
instCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")
codeList = instCpCodeMgr.GetStockListByMarket(1)
# print(codeList)

In [23]:
buyList = []

instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
for code in codeList:
    value = CheckVolumn(instStockChart, code)
    if value == 1:
        buyList.append(code)
    time.sleep(1)

In [24]:
buyList

['A000890',
 'A011280',
 'A017370',
 'A093240',
 'A139290',
 'A139660',
 'A200030',
 'A395760',
 'Q530080',
 'Q570050',
 'Q570057',
 'Q580017',
 'Q580022',
 'Q610010']

In [25]:
kospi = pd.read_csv("kospi_list.csv", encoding='euc-kr')
kospi.head()

,code,name
0,A000020,동화약품
1,A000040,KR모터스
2,A000050,경방
3,A000060,메리츠화재
4,A000070,삼양홀딩스


In [30]:
all_list = pd.DataFrame(buyList)
all_list = all_list.rename(columns ={0:"code"})
all_list.head()

,code
0,A000890
1,A011280
2,A017370
3,A093240
4,A139290


In [32]:
all_list = pd.merge(all_list, kospi, on = 'code', how = 'left')
all_list.head()

,code,name
0,A000890,보해양조
1,A011280,태림포장
2,A017370,우신시스템
3,A093240,형지엘리트
4,A139290,TIGER 200 경기소비재


In [33]:
all_list

,code,name
0,A000890,보해양조
1,A011280,태림포장
2,A017370,우신시스템
3,A093240,형지엘리트
4,A139290,TIGER 200 경기소비재
5,A139660,KOSEF 미국달러선물인버스
6,A200030,KODEX 미국S&P산업재(합성)
7,A395760,ARIRANG ESG성장주액티브
8,Q530080,삼성 인버스 항셍테크 ETN(H)
9,Q570050,TRUE S&P500 선물 ETN
